In [2]:
import os
import mne
import pandas as pd
import numpy as np
import janitor
import warnings
from tqdm._tqdm_notebook import tqdm
import dask.dataframe as dd

import torch
from torch.utils.data import Dataset

warnings.filterwarnings("ignore", category=RuntimeWarning)

data_path = '/home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp'
final_dataset_path = '/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset'

/tmp/ipykernel_86511/1221673159.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm
/home/owner/Documents/DEV/BrainLabyrinth/.venv/lib/python3.10/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# from sklearn.decomposition import PCA as sklearnPCA
import scipy.signal as signal

for user_id in tqdm(os.listdir(data_path)):
    person_dir = os.path.join(data_path, user_id)
    for file in os.listdir(person_dir):
        file_path = os.path.join(person_dir, file)

        if file.endswith(".vhdr"):
            # Read the EEG data
            raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)
            
            # Apply preprocessing steps

            # 1. Resample to 500 Hz
            raw.resample(500, npad="auto")  # Resample to 500 Hz
            
            # 2. Filter high-pass at 0.1 Hz (for low-frequency noise removal)
            raw.filter(l_freq=0.1, h_freq=None)  # High-pass filter (low-frequency cutoff at 0.1 Hz)
            
            # 3. Notch filter at 50 Hz (to remove power line noise)
            raw.notch_filter(freqs=50, picks='all')  # Notch filter for 50 Hz

            # 4. Detrend the data using scipy.signal.detrend
            raw._data = signal.detrend(raw._data, axis=1)  # Detrend along the time axis (axis=1)

            # 5. Apply PCA for dimensionality reduction (if necessary)
            # n_components = 20
            # pca = sklearnPCA(n_components=n_components)
            # data_pca = pca.fit_transform(raw.get_data())  # Apply PCA to the EEG data
            # raw._data = data_pca  # Replace the original data with the PCA-transformed data
            
            # Continue with the rest of your pipeline as you have it
            # Find the corresponding .vmrk file
            vmrk_file = file.replace('.vhdr', '.vmrk')
            vmrk_file_path = os.path.join(person_dir, vmrk_file)

            if os.path.exists(vmrk_file_path):
                # Read the annotations (markers) from the .vmrk file
                annotations = mne.annotations.read_annotations(vmrk_file_path)
            # Convert raw data to DataFrame
            time_series = raw.to_data_frame()

            # Extract markers (annotations)
            marker_times = annotations.onset  # In seconds
            marker_labels = annotations.description  # The marker labels

            # Create a DataFrame for the markers
            markers_df = pd.DataFrame({
                'event_id': np.arange(len(marker_labels)), 
                'start': marker_times - 3.0,
                'end': marker_times + 1.0,
                'marker': marker_labels
            })
            
            markers_df = markers_df[
                ~markers_df.marker.isin([
                    'Marker/Impedance', 'New Segment/', 'Stimulus/2'
                ])
            ]
            # markers_df.marker = markers_df.marker.replace({'Stimulus/2': 'Stimulus/P'})

            # Display the first few rows of both DataFrames
            # Merge markers with EEG data
            time_series['time'] = time_series['time'].round(3)  # Round times to 3 decimal places for matching

            merged_df = janitor.conditional_join(
                markers_df, 
                time_series,
                ('start', 'time', '<='),
                ('end', 'time', '>='),
                how='left',
                df_columns=['event_id', 'marker']
            )
            # display(merged_df.groupby('event_id').count())
            merged_path = os.path.join(final_dataset_path, f"{user_id}.parquet")
            merged_df.to_parquet(merged_path)


  0%|          | 0/53 [00:00<?, ?it/s]

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Filter length: 16501 samples (33.002 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 sa

In [10]:
# Define the directory containing the Parquet files
parquet_directory = '/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset (copy)'

# List all Parquet files in the directory
parquet_files = [os.path.join(parquet_directory, f) for f in os.listdir(parquet_directory) if f.endswith('.parquet')]

# Initialize an empty list to hold the Dask DataFrames
dask_dfs = []

# Process each Parquet file
for file in parquet_files:
    # Read the Parquet file into a Dask DataFrame
    df = dd.read_parquet(file)

    # Extract the file name without the extension
    file_name = os.path.basename(file).replace('.parquet', '')

    # Create a unique event_id by combining the original event_id with the file name
    df['event_id'] = df['event_id'].astype(str) + '_' + file_name

    # Append the DataFrame to the list
    dask_dfs.append(df)

# Concatenate all DataFrames into a single Dask DataFrame
combined_df = dd.concat(dask_dfs, ignore_index=True)

# Create a mapping dictionary for unique event_id values
unique_event_ids = combined_df['event_id'].unique().compute()
event_id_mapping = {event_id: idx for idx, event_id in enumerate(unique_event_ids)}

# Renumber the event_id column using the mapping dictionary
combined_df['event_id'] = combined_df['event_id'].map(event_id_mapping, meta=('event_id', 'int64'))

# Optionally, you can perform operations on the combined Dask DataFrame
# For example, you can compute the first few rows to verify the data
print(combined_df.head())

# Save the combined Dask DataFrame to a new Parquet file
combined_df.to_parquet('/home/owner/Documents/DEV/BrainLabyrinth/data/combined.parquet')


   event_id      marker   time        Fp1        Fpz       Fp2        F7  \
0         0  Stimulus/1  9.438   9.537240   7.803913 -6.957125 -3.662101   
1         0  Stimulus/1  9.440  13.396619  13.868635 -2.922562  3.933463   
2         0  Stimulus/1  9.442  14.850883  14.096507 -0.078051 -0.246867   
3         0  Stimulus/1  9.444  21.293703  18.676068  2.396635  5.013130   
4         0  Stimulus/1  9.446  20.054007  18.219546  3.120409  0.285677   

         F3         Fz         F4  ...        PO3        PO4        PO6  \
0 -1.831665   6.678428   0.643271  ...  11.458283  -8.927917   3.754084   
1  1.559054   9.798244   3.592579  ...  -5.096768  -1.385707  27.073425   
2  1.384730  11.079935   4.932076  ...  -1.438725  -4.064985  22.191193   
3  5.984928  13.828105   6.369710  ...   3.591888 -23.756455 -26.819381   
4  6.550377  17.138199  15.234075  ...  -6.488521 -21.213362 -21.120105   

         FT7       FT8        TP7        TP8        PO7        PO8         Oz  
0   0.357387

In [7]:
import dask.dataframe as dd

# Define the directory containing the Parquet files
parquet_directory = '/home/owner/Documents/DEV/BrainLabyrinth/data/combined.parquet'

# Read the Parquet files into a Dask DataFrame
dask_df = dd.read_parquet(parquet_directory)

# Compute the shape of the Dask DataFrame
rows, cols = dask_df.shape
rows_computed = rows.compute()

# Print the shape
print(f"Shape of the final dataset: ({rows_computed}, {cols})")

# Optionally, you can compute the first few rows to verify the data
print(dask_df.head())


Shape of the final dataset: (14687178, 67)
   event_id      marker   time        Fp1        Fpz       Fp2        F7  \
0         0  Stimulus/1  9.438   9.537240   7.803913 -6.957125 -3.662101   
1         0  Stimulus/1  9.440  13.396619  13.868635 -2.922562  3.933463   
2         0  Stimulus/1  9.442  14.850883  14.096507 -0.078051 -0.246867   
3         0  Stimulus/1  9.444  21.293703  18.676068  2.396635  5.013130   
4         0  Stimulus/1  9.446  20.054007  18.219546  3.120409  0.285677   

         F3         Fz         F4  ...        PO3        PO4        PO6  \
0 -1.831665   6.678428   0.643271  ...  11.458283  -8.927917   3.754084   
1  1.559054   9.798244   3.592579  ...  -5.096768  -1.385707  27.073425   
2  1.384730  11.079935   4.932076  ...  -1.438725  -4.064985  22.191193   
3  5.984928  13.828105   6.369710  ...   3.591888 -23.756455 -26.819381   
4  6.550377  17.138199  15.234075  ...  -6.488521 -21.213362 -21.120105   

         FT7       FT8        TP7        TP8     

In [3]:
import dask.dataframe as dd
import torch
from torch.utils.data import Dataset

# Define the directory containing the Parquet files
parquet_directory = '/home/owner/Documents/DEV/BrainLabyrinth/data/combined.parquet'

class EventDataset(Dataset):
    def __init__(self, path):
        self.__dataframe = dd.read_parquet(path)
        self.__event_ids = self.__dataframe['event_id'].unique().compute()
        self.__len = len(self.__event_ids)

    def __len__(self):
        return self.__len

    def __getitem__(self, idx):
        event_id = self.__event_ids[idx]
        event_rows = self.__dataframe[self.__dataframe['event_id'] == event_id].compute()

        # Convert the event rows to a tensor
        event_tensor = torch.tensor(event_rows.drop(columns=['event_id', 'time', 'marker']).values, dtype=torch.float32)
        marker = torch.tensor(event_rows.marker.values[0] == 'Stimulus/P', dtype=torch.float32)

        return marker, event_tensor

# Create an instance of the EventDataset
dataset = EventDataset(parquet_directory)
marker, features = dataset[0]
# Example usage
print(f"Number of events: {len(dataset)}")
print(f"First event data: {marker.shape}, {features.shape}")
print(marker)


Number of events: 7344
First event data: torch.Size([]), torch.Size([2000, 64])
tensor(0.)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Hyperparameters
input_size = 64  # Number of features
hidden_size = 128  # Number of features in the hidden state
num_layers = 2  # Number of recurrent layers
output_size = 1  # Number of output classes (binary classification)
learning_rate = 0.001
num_epochs = 10
batch_size = 32

# Initialize the model, loss function, and optimizer
model = RNNClassifier(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create a DataLoader for the dataset
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in tqdm(range(num_epochs)):
    for i, (markers, features) in tqdm(enumerate(dataloader)):
        markers = markers.unsqueeze(1)  # Change shape to (batch_size, 1)

        outputs = model(features)
        loss = criterion(outputs, markers)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

print("Training complete.")


  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]

torch.Size([32, 2000, 64])


KeyboardInterrupt: 